In [1]:
%matplotlib notebook 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
Apple = "Resources/AppleStore.csv"
Google = "Resources/googleplaystore.csv"

import seaborn as sns
color = sns.color_palette()
sns.set(rc={'figure.figsize':(25,15)})

import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf


import warnings
warnings.filterwarnings('ignore')
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf


import warnings
warnings.filterwarnings('ignore')

In [2]:
Apple = pd.read_csv(Apple)
Google = pd.read_csv(Google)
#Apple_Clean = Apple[["track_name", "price" ,"rating_count_tot", "user_rating", "prime_genre"]]
#Google_Clean = Google[["App", "Category", "Rating", "Reviews", "Type", "Price", "Genres"]]

In [9]:
google = Google[["App","Category","Rating","Reviews","Size","Installs","Type","Price","Genres"]]

google["Size"] = google["Size"].replace({"M":""}, regex=True)
google["Price"] = google["Price"].replace({"\$":""}, regex=True)
google = google.drop(10472)
google["Price"] = pd.to_numeric(google["Price"]).map("{:.1f}".format)
google = google.rename(columns={"Size":"Size (M)"})
google["Category"] = google["Category"].str.title()
google["Category"] = google["Category"].replace({"_And_":" & ","_":" "},regex=True)
google = google.reset_index(drop=True)

In [10]:
apple = Apple[["track_name","size_bytes","price","rating_count_tot","user_rating","prime_genre"]]

apple = apple.rename(columns = {"track_name":"App","rating_count_tot":"Rating Count",
                                          "user_rating":"Rating","prime_genre":"Genres","price":"Price"})
apple["Size (M)"] = (apple["size_bytes"]*(1e-6)).map("{:.1f}".format)
apple = apple.drop(columns = "size_bytes")
apple["Type"]=""
apple.loc[apple["Price"]==0,["Type"]]="Free"
apple.loc[apple["Type"]=="",["Type"]]="Paid"

In [11]:
google["New Genres"]=""
google.loc[google["Category"]=="Game",["New Genres"]]="Games"

google.loc[(google["Category"]=="Education")|
           (google["Category"]=="Books & Reference")|
           (google["Category"]=="Libraries & Demo"),["New Genres"]]="Education"

google.loc[(google["Category"]=="Entertainment")|
           (google["Category"]=="Events")|
           (google["Category"]=="Social")|
           (google["Category"]=="Sports")|
           (google["Category"]=="News & Magazines"),["New Genres"]]="Entertainment"

google.loc[(google["Category"]=="Video Players")|
           (google["Category"]=="Photography"),["New Genres"]]="Photo & Video"

google.loc[(google["Category"]=="Tools")|
           (google["Category"]=="Communication")|
           (google["Category"]=="Personalization")|
           (google["Category"]=="Weather")|
           (google["Category"]=="Maps & Navigation")|
           (google["Category"]=="Finance"),["New Genres"]]="Utilities"

google.loc[(google["Category"]=="Productivity")|
           (google["Category"]=="Art & Design")|
           (google["Category"]=="Business"),["New Genres"]]="Productivity"

google.loc[(google["Category"]=="Health & Fitness")|
           (google["Category"]=="Medical"),["New Genres"]]="Health & Fitness"

In [12]:
categories = ["Education", "Entertainment", "Health & Fitness", "Games", "Productivity", "Photo & Video", "Utilities"]
games = google.loc[google["New Genres"]=="Games"]["Genres"]

family = google.loc[google["Category"]=="Family"]["Genres"].str.split(";", expand=True)[0].to_frame()

family[0] = family[0].replace({"Educational":"Education"})

for index, row in family.iterrows():
    if row.isin(categories).any():
        google.iloc[index,8]=row[0]
    elif row.isin(games).any():
        google.iloc[index,8]="Games"
    elif row[0] == "Art & Design":
        google.iloc[index,8]="Productivity"
    elif row[0] == "Video Players & Editors":
        google.iloc[index,8]="Photo & Video"
    elif row[0] == "Communication":
        google.iloc[index,8]="Utilities"
    elif row[0] == "Books & Reference":
        google.iloc[index,8]="Education"

google.loc[google["New Genres"]=="",["New Genres"]]="Other"

google_cat = google.groupby("New Genres").count().sort_values("App",ascending=False)

In [13]:
apple["New Genres"]=""
apple.loc[apple["Genres"]=="Games",["New Genres"]]="Games"

apple.loc[(apple["Genres"]=="Education")|
          (apple["Genres"]=="Book")|
          (apple["Genres"]=="Reference"),["New Genres"]]="Education"

apple.loc[(apple["Genres"]=="Entertainment")|
          (apple["Genres"]=="Social Networking")|
          (apple["Genres"]=="Sports")|
          (apple["Genres"]=="News"),["New Genres"]]="Entertainment"

apple.loc[apple["Genres"]=="Photo & Video",["New Genres"]]="Photo & Video"
apple.loc[(apple["Genres"]=="Utilities")|
          (apple["Genres"]=="Weather")|
          (apple["Genres"]=="Navigation")|
          (apple["Genres"]=="Finance"),["New Genres"]]="Utilities"

apple.loc[(apple["Genres"]=="Productivity")|
          (apple["Genres"]=="Business"),["New Genres"]]="Productivity"

apple.loc[(apple["Genres"]=="Health & Fitness")|
          (apple["Genres"]=="Medical"),["New Genres"]]="Health & Fitness"

apple.loc[apple["New Genres"]=="",["New Genres"]]="Other"

apple_cat = apple.groupby("New Genres").count().sort_values("App",ascending=False)

In [14]:
google.head()

,App,Category,Rating,Reviews,Size (M),Installs,Type,Price,Genres,New Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design,4.1,159,19,"10,000+",Free,0.0,Art & Design,Productivity
1,Coloring book moana,Art & Design,3.9,967,14,"500,000+",Free,0.0,Art & Design;Pretend Play,Productivity
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design,4.7,87510,8.7,"5,000,000+",Free,0.0,Art & Design,Productivity
3,Sketch - Draw & Paint,Art & Design,4.5,215644,25,"50,000,000+",Free,0.0,Art & Design,Productivity
4,Pixel Draw - Number Art Coloring Book,Art & Design,4.3,967,2.8,"100,000+",Free,0.0,Art & Design;Creativity,Productivity


In [17]:
# - Installs : Remove + and ,

google['Installs'] = google['Installs'].apply(lambda x: x.replace('+', '') if '+' in str(x) else x)
google['Installs'] = google['Installs'].apply(lambda x: x.replace(',', '') if ',' in str(x) else x)
google['Installs'] = google['Installs'].apply(lambda x: int(x))

#changing the price
google["Price"] = google[google.columns[7]].replace("[\$,]", "", regex=True).astype(float)

In [18]:
bins = [-0.1, 0.0, 1.00, 3.00, 4.00, 5.00, 10.00, 20.00, 50.00, 100.00, 1000.00]
group_labels = ["Free", "<$1", "$1-3", "$3-4", "$4-5",
                "$5-10", "$10-20", "$20-50", "$50-100", "100+"]
apple["Price Category"] = pd.cut(apple["Price"], bins=bins, labels=group_labels, right=True)
google["Price Category"] =  pd.cut(google["Price"], bins=bins, labels=group_labels, right=True)

In [19]:
apple = apple[apple["Price Category"] != "100+"]
google = google[google["Price Category"] != "100+"]
apple = apple[apple["Price Category"] != "$50-100"]
google = google[google["Price Category"] != "$50-100"]

In [32]:
apple.to_csv("Resources/Apple_Data.csv", encoding='utf-8')

In [33]:
google.to_csv("Resources/Google_Data.csv", encoding='utf-8')

In [34]:
google_cat.to_csv("Resources/Google_cat.csv", encoding='utf-8')

In [35]:
apple_cat.to_csv("Resources/Apple_cat.csv", encoding='utf-8')